In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime
from io import StringIO

In [3]:
data_dir = 'Trace_files'
fnames = [x for x in filter(lambda x: x.endswith('.txt'), os.listdir(os.path.join('.', data_dir)))]

In [4]:
fname = fnames[0]
df = pd.read_csv(os.path.join(data_dir, fname), delimiter='\t\t', header = None, dtype = float)

G:\condaInstall\envs\nlp1\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [6]:
request_size = 

'log106_Messaging.txt'